In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# CasheBackedEmbeddings

1. 개요
    - 임베딩 계산 결과를 캐싱 
    - 불필요한 API 호출을 방지하고 성능을 최적화 하기 위한 도구 

2. 주요 특징 
    - 중복 작업 방지:
        - 동일한 텍스트에 대해 이미 게산된 임베딩 결과 재사용 
        - 동일 요청에 대해 API 호출을 반복하지 않음 
    - 성능 최적화:
        - 임베딩 결과를 캐시에서 읽어오기 때문에 처리 속도가 크게 향상 
        - 대규모 데이터나 실시간 응답 속도가 중요한 어플리케이션에서 유용


[Reference] https://python.langchain.com/docs/how_to/caching_embeddings/

In [3]:
import os
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS


In [4]:
# 임베딩 정의
embeddings = OpenAIEmbeddings()

# 로컬 파일 저장소 정의/초기화
store = LocalFileStore('./cache/')

# 캐시 지원하는 임베딩 설정 
# namespace: 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩 생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model, 
)

In [5]:
# 캐싱된 임베딩 확인 
cached_embeddings

In [6]:
# 파일 위치 정의 
file_path = './data/appendix-keywords.txt'

# 문서 로딩 
loader = TextLoader(file_path).load()

# 문서 분할 정의 
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서 분할 
documents = text_splitter.split_documents(loader)

In [7]:
# 벡터스토어 저장 
%time

vector_stores = FAISS.from_documents(documents, cached_embeddings)

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.05 μs


In [8]:
print("캐싱된 임베딩 파일 확인:")
for root, dirs, files in os.walk('./cache/'):
    for file in files:
        print(f" - {os.path.join(root, file)}")

캐싱된 임베딩 파일 확인:
 - ./cache/text-embedding-ada-0020fd71f95-1342-512d-9d5b-3e3ab3c6bbe0
 - ./cache/text-embedding-ada-00274ae75af-9058-555e-aefa-082f0b4e0560
 - ./cache/text-embedding-ada-00241e7391b-b68f-5e9f-bb07-3609bb83c3e2
 - ./cache/text-embedding-ada-0027494a7c8-3399-52a1-85ef-f4d0a563d31f
 - ./cache/text-embedding-ada-0029db9e1cd-62d8-50fc-94f4-24bef3cacaf5
 - ./cache/text-embedding-ada-002cc824f84-d691-544f-9d9c-ca7e45470bb2
 - ./cache/text-embedding-ada-0022112b0ec-6ade-59c9-b09c-755b33c3d32c


In [9]:
print("\n캐싱된 특정 문서 임베딩 확인:")
for doc in documents[:3]:  # 첫 3개의 문서에 대해 확인
    print(f"문서 내용: {doc.page_content[:50]}...")  # 내용 일부 출력
    vector = cached_embeddings.embed_query(doc.page_content)
    print(f"임베딩 벡터 길이: {len(vector)}")
    print(f"임베딩 벡터 예시: {vector[:5]}...\n")  # 일부 벡터 출력


캐싱된 특정 문서 임베딩 확인:
문서 내용: Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을...
임베딩 벡터 길이: 1536
임베딩 벡터 예시: [-0.03266245126724243, -2.554550746936002e-06, 0.007919179275631905, -0.023124800994992256, -0.042786210775375366]...

문서 내용: 정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 ...
임베딩 벡터 길이: 1536
임베딩 벡터 예시: [-0.01606258936226368, -0.006327888462692499, 0.009661508724093437, -0.02922406606376171, -0.0394977368414402]...

문서 내용: 정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하...
임베딩 벡터 길이: 1536
임베딩 벡터 예시: [-0.03586680442094803, 0.0001885255624074489, 0.013284999877214432, -0.007410496473312378, -0.015427306294441223]...



-----
** End of Documents **